In [17]:
"""
This script is made to preprocess raw data from Fluxnet sites. It reads in zipped
raw-data and outputs .csv-files with forcing data to LPJ-GUESS and benchmark-data
for the post-processing script.

Written by: Adrian Gustafson
Date: 2018-06-12
"""
import os
from zipfile import ZipFile
from datetime import datetime

import pandas as pd

In [18]:
def format_dates(row):
    # Convert the Timestamp from double to string for slicing
    timestamp = str(row['TIMESTAMP'].tolist())[:-2]
    row['Year'] = timestamp[:4]
    fmt = '%Y%m%d'
    
    if len(timestamp) == 6:
        # Monthly values
        row['Month'] = timestamp[4:]
    elif len(timestamp) == 8:
        # daily values
        dt = datetime.strptime(timestamp, fmt)
        # Check if the date is a leap year, in that case retun None so that the
        # row can be dropped later.
        if timestamp[4:] == '0229':
            row['Month'] = None
            row['Day'] = None
            row['MonthDay'] = None
            row['DOY'] = None
        else:
            row['Month'] = timestamp[4:6]
            row['Day']   = timestamp[6:]
            row['MonthDay'] = timestamp[4:]
            # LPJ-GUESS day start at 0 but julian days start at 1
            row['DOY'] = int(dt.strftime('%j')) - 1
    elif len(timestamp) == 12:
        fmt = '%Y%m%d%H%M'
        # hourly values
        dt = datetime.strptime(timestamp, fmt)
        # Check if the date is a leap year, in that case retun None so that the
        # row can be dropped later.
        if timestamp[4:] == '0229':
            row['Month'] = None
            row['Day'] = None
            row['MonthDay'] = None
            row['DOY'] = None
            row['Hour'] = None
        else:
            row['Month'] = timestamp[4:6]
            row['Day']   = timestamp[6:]
            row['MonthDay'] = timestamp[4:8]
            # LPJ-GUESS day start at 0 but julian days start at 1
            row['DOY'] = int(dt.strftime('%j')) - 1
            row['Hour'] = int(timestamp[4:8])/30
    else:
        # Should not happen
        print('invalid timeformat at timestamp {}'.format(timestamp))
        
    return row

In [19]:

"""
SETTINGS
""" 
BASE_DIR   = '/Users/quan/projects/MySkill/model_data_fusion/Input' #'/dauta/FLUXNET_from_Michael/fluxnet benchmark/'
INPUT_DIR  = 'zip'
OUTPUT_DIR = 'US-Me2'

sitelist_fn = 'fluxnet2015_sitelist.csv'
LAI = 6.2

# 
# forcing_cols   = ['TA_ERA', 'SW_IN_ERA', 'P_ERA']
# benchmark_cols = ['NEE_VUT_REF', 'GPP_NT_VUT_REF', 'LE_F_MDS']

forcing_cols   = ['TA_ERA', 'SW_IN_ERA', 'SWC_F_MDS_1','VPD_ERA','P_ERA', 'NETRAD', 'LE_F_MDS']
benchmark_cols = ['NEE_VUT_REF', 'GPP_NT_VUT_REF', 'LE_F_MDS']
input_cols = ['TIMESTAMP'] + forcing_cols + benchmark_cols

# Rename input variables according to this map.

# '',

column_map = {
    'TA_ERA': 'TEMP',
    'SW_IN_ERA': 'Rad',
        
    'SWC_F_MDS_1': 'SOILM',
    'VPD_ERA': 'VPD',
    'P_ERA': 'P', 
    'NETRAD': 'RNET', 
    'LE_F_MDS': 'ET',
    
    'NEE_VUT_REF': 'NEE',
    'GPP_NT_VUT_REF': 'GPP',
    'Lon': 'Lon',
    'Lat': 'Lat',
    'IGBP': 'IGBP',
    'DOY': 'DOY'
}

In [20]:
"""
SCRIPT STARTS
"""

sitelist = pd.read_csv(os.path.join(BASE_DIR, sitelist_fn), encoding = "ISO-8859-1")

daily = pd.DataFrame()
monthly = pd.DataFrame()
hourly = pd.DataFrame()
for fn in os.listdir(os.path.join(BASE_DIR, INPUT_DIR)):
    site = fn.split('_')[1]
    print('Extracting data from site {}'.format(site))
    site_data = sitelist[sitelist['SITE_ID'] == site]
    try:
        zf = ZipFile(os.path.join(BASE_DIR, INPUT_DIR, fn))
    except:
        print('Bad file at site {}'.format(site))
        continue
        
    for f in zf.namelist():
        if '_FULLSET_DD_' in f :
            daily_raw = f
            
        if '_FULLSET_MM_' in f :
            monthly_raw = f
        if '_FULLSET_HH_' in f :
            hourly_raw = f
        
    df_daily = pd.read_csv(zf.open(daily_raw), usecols=input_cols)
    df_monthly = pd.read_csv(zf.open(monthly_raw), usecols=input_cols)
    df_hourly = pd.read_csv(zf.open(hourly_raw), usecols=['TIMESTAMP_START'] + forcing_cols + benchmark_cols)
    df_hourly = df_hourly.rename(columns={'TIMESTAMP_START':'TIMESTAMP'})
    
    df_daily = df_daily.apply(format_dates, axis=1)
    df_monthly = df_monthly.apply(format_dates, axis=1)
    df_hourly = df_hourly.apply(format_dates, axis=1)

    # Drop 29th of February
    df_daily.dropna(inplace=True)
    df_monthly.dropna(inplace=True)
    df_hourly.dropna(inplace=True)

    
    lon = site_data['LOCATION_LONG'].values[0]
    lat = site_data['LOCATION_LAT'].values[0]
    igbp = site_data['IGBP'].values[0]
    
    df_daily['Lon'] = lon; df_daily['Lat'] = lat; df_daily['IGBP'] = igbp
    df_monthly['Lon'] = lon; df_monthly['Lat'] = lat; df_monthly['IGBP'] = igbp
    df_hourly['Lon'] = lon; df_hourly['Lat'] = lat; df_hourly['IGBP'] = igbp

    if len(df_daily) % 365 != 0:
        print('Skipping location at site {}. Not full years'.format(site))
        continue
    
    daily = pd.concat([daily, df_daily[['Lon', 'Lat', 'Year', 'DOY', 'IGBP']+benchmark_cols]], ignore_index=True)
    monthly = pd.concat([monthly, df_monthly[['Lon', 'Lat', 'Year', 'Month', 'IGBP']+benchmark_cols]], ignore_index=True)
    hourly = pd.concat([hourly, df_hourly[['Lon', 'Lat', 'Year', 'Month', 'IGBP']+benchmark_cols]], ignore_index=True)

    # df_daily[['Year', 'MonthDay']+forcing_cols].to_csv(os.path.join(BASE_DIR, OUTPUT_DIR, '{}.csv'.format(site)),
    #                                                     sep='\t',
    #                                                     index=False, #header=False,
    #                                                     float_format='%.3f')
    
daily.rename(columns=column_map, inplace=True)
monthly.rename(columns=column_map, inplace=True)
hourly.rename(columns=column_map, inplace=True)

daily.to_csv(os.path.join(BASE_DIR, OUTPUT_DIR, 'daily.csv'), sep='\t', index=False)
monthly.to_csv(os.path.join(BASE_DIR, OUTPUT_DIR, 'monthly.csv'), sep='\t', index=False)
hourly.to_csv(os.path.join(BASE_DIR, OUTPUT_DIR, 'monthly.csv'), sep='\t', index=False)

Extracting data from site US-Me2


In [93]:
df_hourly_for_AMIS = df_hourly[['Year', 'MonthDay','DOY']+forcing_cols+benchmark_cols].rename(columns=column_map)

df_hourly_for_AMIS['SOILM2'] = df_hourly_for_AMIS['SOILM'] 
df_hourly_for_AMIS['LAI'] = LAI
df_hourly_for_AMIS['Vegk'] = 1 # Vegk the canopy extinction coefficient calculated using solar zenith angle assuming a spherical leaf angle distribution

df_hourly_for_AMIS['GA'] = 1
df_hourly_for_AMIS['GA_U'] = 1
df_hourly_for_AMIS['GSOIL'] = 1

df_hourly_for_AMIS.to_csv(os.path.join(BASE_DIR, '{}.csv'.format(site)),
                        index=False, #header=False,
                        float_format='%.3f')

print('Finished!')

Finished!
